In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy.misc import toimage
import skimage.color
from torch import autograd
from torch.autograd import Variable

device = torch.device('cuda')     # Default CUDA device

### Parameters

In [2]:
SIZE = 512
BETA1 = 0.5
BETA2 = 0.999
LAMBDA = 10
ALPHA = 1000
BATCH_SIZE = 5
NUM_EPOCHS = 5
LATENT_DIM = 100
TRAIN_NUM = 50

### Generator Network

In [3]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        #  Convolutional layers 
        
        # input 512x512x3  output 512x512x16
        self.conv1 = nn.Conv2d(3, 16, 5, stride = 1, padding = 2)
        self.conv1_bn = nn.BatchNorm2d(16)
        
        # input 512x512x16  output 256x256x32
        self.conv2 = nn.Conv2d(16, 32, 5, stride = 2, padding = 2)
        self.conv2_bn = nn.BatchNorm2d(32)
        
        # input 265x256x32  output 128x128x64
        self.conv3 = nn.Conv2d(32, 64, 5, stride = 2, padding = 2)
        self.conv3_bn = nn.BatchNorm2d(64)
        
        # input 128x128x64  output 64x64x128
        self.conv4 = nn.Conv2d(64, 128, 5, stride = 2, padding = 2)
        self.conv4_bn = nn.BatchNorm2d(128)
        
        # input 64x64x128  output 32x32x128
        # the output of this layer we need layers for global features
        self.conv5 = nn.Conv2d(128, 128, 5, stride = 2, padding = 2)
        self.conv5_bn = nn.BatchNorm2d(128)
        
        # convs for global features
        # input 32x32x128 output 16x16x128
        self.conv51 = nn.Conv2d(128,128,5, stride =2 , padding =2 )
        
        # input 16x16x128 output 8x8x128
        self.conv52 = nn.Conv2d(128,128,5, stride =2 , padding =2 )
        
        # input 8x8x128 output 1x1x128
        self.conv531 = nn.Conv2d(128,128,5, stride =2 , padding =1 )
        
        # input 1x1x128 output 1x1x128
        self.conv532 = nn.Conv2d(128,128,5, stride =2 , padding =1 )
        
        # input 32x32x128 output 32x32x128
        # the global features should be concatenated to the feature map aftere this layer
        # the output after concat would be 32x32x256
        self.conv6 = nn.Conv2d(128, 128, 5, stride = 1, padding =2)
        
        # input 32x32x256 output 32x32x128
        self.conv7 = nn.Conv2d(256, 128, 5, stride = 1, padding = 2)
        
        # deconvolutional layers
        # input 32x32x128 output 64x64x128
        self.dconv1 = nn.ConvTranspose2d(128, 128, 5, stride = 2, padding = 2, output_padding = 1)
        self.dconv1_bn = nn.BatchNorm2d(128)
        
        # input 64x64x256 ouput 128x128x128
        self.dconv2 = nn.ConvTranspose2d(256, 128, 5, stride = 2, padding = 2, output_padding = 1)
        self.dconv2_bn = nn.BatchNorm2d(256)
        
        # input 128x128x192 output 256x256x64
        self.dconv3 = nn.ConvTranspose2d(192, 64, 5, stride = 2, padding = 2, output_padding = 1)
        self.dconv3_bn = nn.BatchNorm2d(192)
        
        # input 256x256x96 ouput 512x512x32
        self.dconv4 = nn.ConvTranspose2d(96, 32, 5, stride = 2, padding = 2, output_padding = 1)
        self.dconv4_bn = nn.BatchNorm2d(96)
        
        # final convolutional layers
        # input 512x512x48 output 512x512x16
        self.conv8 = nn.Conv2d(48, 16, 5, stride = 1, padding = 2)
        self.conv8_bn = nn.BatchNorm2d(48)
        
        # input 512x512x16 output 512x512x3
        self.conv9 = nn.Conv2d(16, 3, 5, stride = 1, padding = 2)    
        self.conv9_bn = nn.BatchNorm2d(16)
        # SELU
                
    def forward(self, x):
        # input 512x512x3 to output 512x512x16
        x = self.conv1_bn(F.selu(self.conv1(x)))
        print("x")
        print(x.shape)
        # input 512x512x16 to output 256x256x32
        x1 = self.conv2_bn(F.selu(self.conv2(x)))
        print("x1")
        print(x1.shape)
        # input 256x256x32 to output 128x128x64
        x2 = self.conv3_bn(F.selu(self.conv3(x1)))
        print("x2")
        print(x2.shape)
        # input 128x128x64 to output 64x64x128
        x3 = self.conv4_bn(F.selu(self.conv4(x2)))
        print("x3")
        print(x3.shape)
        # input 64x64x128 to output 32x32x128
        x4 = self.conv5_bn(F.selu(self.conv5(x3)))
        print("x4")
        print(x4.shape)
        #convolutions for global features
        # input 32x32x128 to output 16x16x128
        x51 = self.conv51(x4)
        print("x51")
        print(x51.shape)
        # input 16x16x128 to output 8x8x128
        x52 = self.conv52(x51)
        print("x52")
        print(x52.shape)
        # input 8x8x128 to output 1x1x128
        x53 = self.conv532(F.selu(self.conv531(x52)))
        print("x53")
        print(x53.shape)
        x53_temp = torch.cat([x53]*32,dim = 2 )
        x53_temp = torch.cat([x53_temp]*32,dim=3)
        print("x53_temp")
        print(x53_temp.shape)
        
        # input 32x32x256 to output 32x32x128
        x5 = self.conv6(x4)
        print("x5")
        print(x5.shape)
        # input 32x32x128 to output 32x32x128
        x5 = self.conv7(torch.cat([x5,x53_temp],dim=1))
        print("x5")
        print(x5.shape)
        # input 32x32x128 to output 64x64x128
        xd = self.dconv1(self.dconv1_bn(F.selu(x5)))
        print("xd1")
        print(xd.shape)
        # input 64x64x256 to output 128x128x128
        xd = self.dconv2(self.dconv2_bn(F.selu(torch.cat([xd,x3], dim=1))))
        print("xd2")
        print(xd.shape)
        # input 128x128x192 to output 256x256x64
        xd = self.dconv3(self.dconv3_bn(F.selu(torch.cat([xd,x2],dim=1))))
        print("xd3")
        print(xd.shape)
        # input 256x256x64 to output 512x512x32
        xd = self.dconv4(self.dconv4_bn(F.selu(torch.cat([xd,x1],dim=1))))
        print("xd4")
        print(xd.shape)
        # input 512x512x48 to output 512x512x16
        xd = self.conv8(self.conv8_bn(F.selu(torch.cat([xd,x],dim=1))))
        print("xd 8")
        print(xd.shape)
        # input 512x512x16 to output 512x512x3
        xd = self.conv9(self.conv9_bn(F.selu((xd))))
        print("xd 9")
        print(xd.shape)
        return xd

### Discriminator Network

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        #  Convolutional layers 
        
        # input 512x512x3  output 512x512x16
        self.conv1 = nn.Conv2d(3, 16, 5, stride = 2, padding = 2)
        self.conv1_in = nn.InstanceNorm2d(16)
        
        # input 512x512x16  output 256x256x32
        self.conv2 = nn.Conv2d(16, 32, 5, stride = 2, padding = 2)
        self.conv2_in = nn.InstanceNorm2d(32)
        
        # input 265x256x32  output 128x128x64
        self.conv3 = nn.Conv2d(32, 64, 5, stride = 2, padding = 2)
        self.conv3_in = nn.InstanceNorm2d(64)
        
        # input 128x128x64  output 64x64x128
        self.conv4 = nn.Conv2d(64, 128, 5, stride = 2, padding = 2)
        self.conv4_in = nn.InstanceNorm2d(128)
        
        # input 64x64x128  output 32x32x128
        # the output of this layer we need layers for global features
        self.conv5 = nn.Conv2d(128, 128, 5, stride = 2, padding = 2)
        self.conv5_in = nn.InstanceNorm2d(128)
        
        # input 32x32x128  output 16x16x128
        # the output of this layer we need layers for global features
        self.conv6 = nn.Conv2d(128, 128, 5, stride = 2, padding = 2)
        self.conv6_in = nn.InstanceNorm2d(128)
        
        # input 16x16x128  output 1x1x1
        # the output of this layer we need layers for global features
        self.conv6 = nn.Conv2d(128, 1, 5, stride = 32, padding = 1)
        self.conv6_in = nn.InstanceNorm2d(1)
        
    def forward(self, x):
        
        # input 512x512x3 to output 512x512x16
        x = self.conv1_in(F.leaky_relu(self.conv1(x)))
        
        # input 512x512x16 to output 256x256x32
        x = self.conv2_in(F.leaky_relu(self.conv2(x)))
        
        # input 256x256x32 to output 128x128x64
        x = self.conv3_in(F.leaky_relu(self.conv3(x)))
        
        # input 128x128x64 to output 64x64x128
        x = self.conv4_in(F.leaky_relu(self.conv4(x)))
        
        # input 64x64x128 to output 32x32x128
        x = self.conv5_in(F.leaky_relu(self.conv5(x)))
        
        # input 32x32x128 to output 16x16x128
        x = self.conv5_in(F.leaky_relu(self.conv5(x)))
        
        # input 16x16x128 to output 1x1x1
        x = self.conv5_in(F.leaky_relu(self.conv5(x)))
        
        return x

In [5]:
generator1 = Generator()
# generator2 = Generator()
discriminator = Discriminator()
# print(generator1)
# print(discriminator)

### Loading Training and Test Set Data

In [6]:
# Converting the images for PILImage to tensor, so they can be accepted as the input to the network
transform = transforms.Compose ([transforms.Resize(SIZE, interpolation=2),transforms.ToTensor()])

trainset = torchvision.datasets.STL10(root='./data', split='unlabeled', transform=transform, target_transform=None, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified


In [7]:
testset = torchvision.datasets.STL10(root='./data', split='test', transform=transform, target_transform=None, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified


### MSE Loss and Optimizer

In [8]:
criterion = nn.MSELoss()

optimizer_g1 = optim.Adam(generator1.parameters(), lr = 0.001, betas=(BETA1,BETA2))
# optimizer_g2 = optim.Adam(generator2.parameters(), lr = 0.001, betas=(BETA1,BETA2))
optimizer_d = optim.Adam(discriminator.parameters(), lr = 0.001, betas=(BETA1,BETA2))

# Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
Tensor = torch.FloatTensor

### Gradient Penalty
Computes gradient penalty loss for A-WGAN

In [9]:
def computeGradientPenalty(D, realSample, fakeSample):
    alpha = Tensor(np.random.random((realSample.size(0), 1, 1, 1)))
    interpolate = alpha * realSample + ((1 - alpha) * fakeSample)
    interpolate = autograd.Variable(interpolate, requries_grad = True)
    dInterpolation = D(interpolate)
    fakeOutput = Variable(Tensor(realSample.shape[0], 1).fill_(1.0), requires_grad=False)
    
    gradients = autograd.grad(
        outputs = dInterpolation,
        inputs = interpolate,
        grad_outputs = fakeOutput,
        create_graph = True,
        retain_graph = True,
        only_inputs = True)[0]
    
    gradients = gradients.view(gradients.size(0), -1)
    gradientPenalty = ((0,(gradients.norm(2, dim=1)-1)).max()).mean()
    return gradientPenalty

### Generator Loss

In [10]:
def generatorAdversarialLoss( output_images):
    validity = discriminator(output_images)
    gen_adv_loss = torch.mean(validity)
    
    return gen_adv_loss

# def computeGeneratorLoss(inputs, outputs_g1, outputs_g2):
def computeGeneratorLoss(inputs, outputs_g1):
    # generator 1
    gen_adv_loss1 = generatorAdversarialLoss(outputs_g1)
    
#     generator 2
#     gen_adv_loss2 = generatorAdversarialLoss(outputs_g2)
    
    i_loss = criterion(inputs, outputs_g1)
    gen_loss = -gen_adv_loss1 + ALPHA*i_loss
    
    return gen_loss



### Discriminator Loss

In [11]:
def discrminatorLoss(d1Real, d1Fake, gradPenalty):
    return (torch.mean(d1Fake) - torch.mean(d1Real.mean)) + (LAMBDA*gradPenalty)

### Training Network

In [12]:
batches_done = 0
for epoch in range(NUM_EPOCHS):
    for i, data in enumerate(trainloader, 0):
        input, dummy = data
        realImgs = Variable(input.type(Tensor))
        noiseImgs = Variable(Tensor(np.random.normal(loc = 0,scale = 1, size = input.shape)))
#         print(noiseImgs)
        ### TRAIN DISCRIMINATOR
        optimizer_d.zero_grad()
        fakeImgs = generator1(noiseImgs)
#         fake2Imgs = generator2(fake1Imgs)
        
        # Real Images
        realValid = discriminator(realImgs)
        # Fake Images
        fakeValid = discriminator(fakeImgs)
        
        gradientPenalty = computeGradientPenalty(discriminator, realImgs.data, fakeImgs.data)
        dLoss = discriminatorLoss(realValid, fakeValid, gradientPenalty)
        dLoss.backward()
        optimizer_d.step()
        optimizer_g1.zero_grad()
        
        ### TRAIN GENERATOR
        if i % TRAIN_NUM == 50:
            print("Hello" + i)
             # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(noiseImgs)
            g_loss = computeGeneratorLoss(noiseImgs,fake_imgs)

            g_loss.backward()
            optimizer_G.step()

            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, NUM_EPOCHS , i, len(dataloader), d_loss.item(), g_loss.item())
            )

            if batches_done % 200 == 0:
                save_image(fake_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

            batches_done += TRAIN_NUM
            
            

x
torch.Size([5, 16, 512, 512])
x1
torch.Size([5, 32, 256, 256])
x2
torch.Size([5, 64, 128, 128])
x3
torch.Size([5, 128, 64, 64])
x4
torch.Size([5, 128, 32, 32])
x51
torch.Size([5, 128, 16, 16])
x52
torch.Size([5, 128, 8, 8])
x53
torch.Size([5, 128, 1, 1])
x53_temp
torch.Size([5, 128, 32, 32])
x5
torch.Size([5, 128, 32, 32])
x5
torch.Size([5, 128, 32, 32])
xd1
torch.Size([5, 128, 64, 64])
xd2
torch.Size([5, 128, 128, 128])
xd3
torch.Size([5, 64, 256, 256])
xd4
torch.Size([5, 32, 512, 512])
xd 8
torch.Size([5, 16, 512, 512])
xd 9
torch.Size([5, 3, 512, 512])


NameError: name 'autograd' is not defined